In [1]:
import cv2
import numpy as np 

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
import tensorflow as tf

In [4]:
#download the file and upload it to your local environment to use it
#loading the face detector xml file

face_detect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
test = face_detect.load('haarcascade_frontalface_default.xml')

print(test)

True


In [5]:
model = tf.keras.models.load_model('modelforemotions.h5')

In [6]:
index_to_emotion = {0: 'angry',
                    1: 'disgusted',
                    2: 'fearful',
                    3: 'happy',
                    4: 'neutral',
                    5: 'sad',
                    6: 'surprised'}



In [7]:
#standard method to start video input via webcam using opencv
video= cv2.VideoCapture(0)
while True:
    ret,img = video.read()

#using .detectMultiScale to extract the (x,y,width,height) of the rectangle
    faces = face_detect.detectMultiScale(img, 1.3, 5)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
#getting the region of interest,which is the face of the person in the image
    for (x,y,width,height) in faces:
        img=cv2.rectangle(img,(x,y),(x+width,y+height),(0,0,255),2)
        roi_gray = gray[y:y+ height,x:x+width]
        roi_color = img[y:y+height, x:x+width]
        image = np.zeros_like(roi_color)
        image[:, :, 0] = roi_gray
        image[:, :, 1] = roi_gray
        image[:, :, 2] = roi_gray
        
        # resizing the image
        try:
            image1 = cv2.resize(image, (48,48), interpolation=cv2.INTER_AREA)
        except Exception as e:
            print(str(e))
        img_fed = np.expand_dims(image1, axis=0)
        scores = model.predict(img_fed)
        index = np.argmax(scores)
        img=cv2.putText(img, index_to_emotion[index], (x,y-15), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), thickness=2)

        
    cv2.imshow('img',img)


#press q key on keyboard to close the webcam videocapture
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break

video.release()
cv2.destroyAllWindows()
    

